## 자연어 처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Oshima
- 부동산 사건 사고가 표기된 info 컬럼 데이터의 주요 키워드 추출  

In [2]:
import pandas as pd

In [5]:
colab_path = '/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing/yunyoung/data'
vscode_path = './data'
open = pd.read_csv(colab_path + '/open_data_add_latlng.csv', index_col=0)
airbnb = pd.read_csv(colab_path+'/airbnb_data_add_latlng.csv', index_col=0)
oshima_data = pd.read_csv(colab_path+'/oshima_data_add_latlng.csv', index_col=0)
oshima_data.head()

,district,address,info,date,full name,latitude,longitude
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


In [ ]:
oshima_data['info'].value_counts()[-10:]

二階、階段をあがって二番目の部屋\n生活保護者の孤高死                                                                        1
60代男性が孤高死                                                                                          1
建築完成後入居前、飛び降り自殺                                                                                    1
50代男性が病死                                                                                           1
桐ヶ丘中学校男子生徒飛び降り自殺                                                                                   1
1階 角部屋 南向き1K（洋室6帖、K1帖） 専有面積15.5㎡\n告知事項あり\n生活保護相談（60代まで）                                            1
やよい荘１０１号室 死体発見（死後３週間前後と推定）                                                                         1
賃貸一戸建て\n鉄骨造 2階建（屋上あり） 主要採光面：南向き5DK（ DK8帖、洋室9帖、洋室4帖、和室6畳、和室6畳、和室6畳） 専有面積135.12㎡\n告知事項有\n定期借家（1年）    1
死体発見 ．                                                                                             1
多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅                                                          

### 데이터 살펴보기

In [ ]:
import ast
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

In [ ]:
pip install ipadic==1.0.0a2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 13.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0a2-py3-none-any.whl size=13556740 sha256=d3048b4ebfcc6a83e236b3142eb25f7b56ed09333b8e7d19b3a4d52f3b777f8a
  Stored in directory: /Users/yunyoung/Library/Caches/pip/wheels/44/f9/02/862651969e5626d2fcc219224a366568ca5cce1eb70c2a48b4
Successfully built ipadic
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.font_manager as font_manager

font_list = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
font_list[:10]

['/System/Library/Fonts/Supplemental/NotoSansMarchen-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Galvji.ttc',
 '/Library/Fonts/Arial Unicode.ttf',
 '/System/Library/Fonts/Supplemental/SignPainter.ttc',
 '/System/Library/Fonts/Supplemental/DevanagariMT.ttc',
 '/System/Library/Fonts/Supplemental/NotoSansImperialAramaic-Regular.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansBhaiksuki-Regular.ttf',
 '/Users/yunyoung/Library/Fonts/NanumBrush.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansSiddham-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Times New Roman Bold.ttf']

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import MeCab

m = MeCab.Tagger()


def extract_nouns(text):
    node = m.parseToNode(text)
    nouns = []
    while node:
        if node.feature.split(',')[0] == '名詞':  # Extract only nouns
            nouns.append(node.surface)
        node = node.next
    return nouns

# Apply the function to each row in the 'info' column
oshima_data['nouns'] = oshima_data['info'].apply(extract_nouns)
oshima_data[:10]

,district,address,info,date,full name,latitude,longitude,nouns
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,"[死体, 発見]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,[自殺]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,"[2, 階, サウナ, ロイヤル, 死体, 発見]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,[自殺]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,"[首吊り, 自殺]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,"[20, 階, 心理, 瑕疵, あり]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,"[コイン, ロッカー, 乳児, 腐乱, 死体]"
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,"[902, 号, 自殺, あり]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,"[伊勢丹, テナント, 関係, 首吊り, 自殺]"
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,"[309, 号, 首吊り, 自殺]"


### 일본어 -> 한국어 번역

In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ja', target='ko')

def translate(x):
    translated_text = translator.translate(x)
    return translated_text

In [ ]:
from tqdm.notebook import tqdm_notebook

tqdm_notebook()

oshima_data['translate'] = ""

for i, row in tqdm_notebook(oshima_data.iterrows(), total=len(oshima_data)):
    oshima_data['translate'][i] = translate(row['info'])

0it [00:00, ?it/s]

  0%|          | 0/4833 [00:00<?, ?it/s]

<ipython-input-7-12f0b8a1dd51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oshima_data['translate'][i] = translate(row['info'])


In [ ]:
oshima_data.to_csv('translate.csv')


### 태깅하기

In [2]:
cd /content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing

/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing


In [3]:
import pandas as pd

oshima_kr = pd.read_csv('./yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [4]:
oshima_kr_unique = oshima_kr.drop_duplicates(subset=['address'])

In [5]:
oshima_kr_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3644 entries, 0 to 4836
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   3644 non-null   object 
 1   address    3644 non-null   object 
 2   info       3644 non-null   object 
 3   date       3644 non-null   object 
 4   full name  3644 non-null   object 
 5   latitude   3644 non-null   float64
 6   longitude  3644 non-null   float64
 7   translate  3644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 256.2+ KB


In [6]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


In [7]:
df_unique['info'].unique()

array(['死体発見', '飛び降り自殺', '2階サウナロイヤル 死体発見', ..., '自殺&死体発見',
       '多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅',
       '多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅'], dtype=object)

In [8]:
# 전처리 함수
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
\
    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens


In [9]:
# !git clone https://github.com/uoneway/KoBertSum.git


Cloning into 'KoBertSum'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 908 (delta 108), reused 102 (delta 102), pack-reused 790
Receiving objects: 100% (908/908), 15.97 MiB | 13.02 MiB/s, done.
Resolving deltas: 100% (576/576), done.


### Korean News Summarization Model : https://huggingface.co/gogamza/kobart-summarization

In [15]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


In [16]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


In [19]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [23]:
df_unique['translate'][0]

'시체 발견'

In [24]:
raw_input_ids = tokenizer.encode(df_unique['translate'][0])
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]


In [25]:
summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'시체 발견으로 시체 처리장치가 작동되는 시체 발견으로 시체 발견'

In [31]:
cd /content

/content


https://github.com/kiyoungkim1/LMkor

In [ ]:
!git clone https://github.com/kiyoungkim1/LMkor
!pip3 install -q transformers

from LMkor.examples.bertshared_summarization import Summarize
summarize = Summarize('kykim/bertshared-kor-base')

In [33]:
text = '''
LG전자가 스마트폰을 담당하는 MC(모바일커뮤니케이션)사업부 분할 및 매각을 위한 법률 자문 업무를 김앤장법률사무소에 맡겼다. MC사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.
22일 인수합병(M&A)업계에 따르면 LG전자는 최근 MC사업부 분할 후 매각 방안 등을 포괄적으로 검토하기 위해 김앤장을 법률자문사로 선임한 것으로 알려졌다. 회계·실사 자문은 EY한영회계법인에 맡길 가능성이 큰 것으로 전해졌다. 김앤장 등 자문사들은 사업본부를 분할한 뒤 사업양수도나 분할사업부의 지분 매각, 지식재산권(IP) 매각 등을 놓고 검토에 들어간 것으로 알려졌다.
업계에서는 LG전자가 MC사업본부를 통매각하기보다는 ‘쪼개기 매각’에 나설 것으로 보고 있다. 스마트폰 선행기술 연구개발(R&D) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측하고 있다. 앞서 권봉석 LG전자 사장은 사내 메시지를 통해 임직원에게 “현재 모든 가능성을 열어 두고 사업 운영방향을 면밀히 검토하고 있다”고 밝히며 매각 추진을 암시했다. M&A업계 관계자는 “거래가 성사되기도 전에 사업 전면 재검토를 공식화한 것은 상당히 이례적”이라며 “향후 매각이 잘 이뤄지지 않더라도 모바일 사업을 철수하겠다는 배수진을 둔 것으로 보인다”고 설명했다.
다만 원매자를 찾기가 쉽지 않을 것이란 전망이 우세하다. LG전자 모바일 사업은 한때 글로벌시장에서 톱5 안에 드는 기술력을 인정받았지만 누적 적자만 5조원에 달하고 있다. 업계에서 평가하는 MC사업부의 가치도 5000억원대에서 수조원대까지 편차가 상당히 크다.
상대적으로 해외 원매자들의 인수의사가 더 확실한 것으로 알려지고 있다. 북미사업 등 글로벌 시장 확장을 원하는 후발기업들이 주요 대상이다. 베트남의 빈그룹과 중국 기업 등이 유력하게 거론된다. 증권업계를 중심으로는 스마트 기기를 연결하는 사물인터넷(IoT) 사업을 염두에 둔 구글, 페이스북 같은 미국 정보기술(IT) 기업들도 원매자 후보군으로 꼽고 있다.
'''

summarize(text)

22일 인수합병 ( m & a ) 업계에 따르면 스마트폰 선행기술 연구개발 ( r & d ) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측되고 있으며 업계에서는 mc사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.


In [36]:
print(df_unique['translate'][5])
summarize(df_unique['translate'][5])

20층
심리적 하자 있음
20


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1279: UserWarning: Unfeasible length constraints: `min_length` (10) is larger than the maximum possible length (2). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


### KeyBert

In [37]:
!pip install keybert
!pip install kiwipiepy


  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=706aeaac485b00591e0695153ff6f188629452808deaa3f444db51cda78886ad
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
Successfully built keybert
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=d15f447bafd113751d1e73c500e34c502f373de9112892e6b39c4e8cd941247a
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [39]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

text="""
김영하가 『살인자의 기억법』 이후 9 년 만에 내놓는 장편소설 『작별인사』는 그리 멀지 않은 미래를 배경으로, 별안간 삶이 송두리째 뒤흔들린 한 소년의 여정을 좇는다.
유명한 IT 기업의 연구원인 아버지와 쾌적하고 평화롭게 살아가던 철이는 어느날 갑자기 수용소로 끌려가 난생처음 날것의 감정으로 가득한 혼돈의 세계에 맞닥뜨리게 되면서 정신적, 신체적 위기에 직면한다.
동시에 자신처럼 사회에서 배제된 자들을 만나 처음으로 생생한 소속감을 느끼고 따뜻한 우정도 싹틔운다. 철이는 그들과 함께 수용소를 탈출하여 집으로 돌아가기 위해 길을 떠나지만 그 여정에는 피할 수 없는 질문이 기다리고 있다.
세계보건기구 WHO가 팬데믹을 선언한 지 2년이 지나서야 작가는 『작별인사』의 개작을 마쳤다. 420매 분량이던 원고는 약 800매로 늘었고, 주제도 완전히 달라졌다.
‘인간을 인간으로 만드는 것은 무엇인가?’, ‘인간과 인간이 아닌 존재들을 가르는 경계는 어디인가’를 묻던 소설은 ‘삶이란 과연 계속될 가치가 있는 것인가?’,
‘세상에 만연한 고통을 어떻게 하면 줄일 수 있을 것인가’, ‘어쩔 수 없이 태어났다면 어떻게 살고 어떻게 죽어야 할 것인가’와 같은 질문을 던지는 이야기로 바뀌었다.
팬데믹이 개작에 영향을 주었을 수도 있고, 원래 『작별인사』의 구상에 담긴 어떤 맹아가 오랜 개작을 거치며 발아했는지도 모른다. 그것에 대해 작가는 이렇게 말하고 있다.
마치 제목이 어떤 마력이 있어서 나로 하여금 자기에게 어울리는 이야기로 다시 쓰도록 한 것 같은 느낌이다. 탈고를 하고 얼마 지나지 않아 원고를 다시 읽어보았다.
이제야 비로소 애초에 내가 쓰려고 했던 어떤 것이 제대로, 남김 없이 다 흘러나왔다는 생각이 들었다. _’작가의 말’에서
전면적인 수정을 통해 2022년의 『작별인사』는 2020년의 『작별인사』를 마치 시놉시스나 초고처럼 보이게 할 정도로 확연하게 달라졌다. 그리고 김영하의 이전 문학 세계와의 연결점들이 분명해졌다.
"""


In [40]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
keywords

[('장편소설', 0.531),
 ('소설은', 0.5247),
 ('위기에', 0.5216),
 ('삶이란', 0.5162),
 ('삶이', 0.5157),
 ('작가는', 0.5111),
 ('정신적', 0.5075),
 ('생생한', 0.5043),
 ('문학', 0.5007),
 ('연구원인', 0.4899)]

In [41]:
kiwi = Kiwi()
kiwi.analyze(text)

[([Token(form='김영하', tag='NNP', start=1, len=3),
   Token(form='가', tag='JKS', start=4, len=1),
   Token(form='『', tag='SSO', start=6, len=1),
   Token(form='살인자', tag='NNG', start=7, len=3),
   Token(form='의', tag='JKG', start=10, len=1),
   Token(form='기억', tag='NNG', start=12, len=2),
   Token(form='법', tag='NNG', start=14, len=1),
   Token(form='』', tag='SSC', start=15, len=1),
   Token(form='이후', tag='NNG', start=17, len=2),
   Token(form='9', tag='SN', start=20, len=1),
   Token(form='년', tag='NNB', start=22, len=1),
   Token(form='만', tag='NNB', start=24, len=1),
   Token(form='에', tag='JKB', start=25, len=1),
   Token(form='내놓', tag='VV', start=27, len=2),
   Token(form='는', tag='ETM', start=29, len=1),
   Token(form='장편', tag='NNG', start=31, len=2),
   Token(form='소설', tag='NNG', start=33, len=2),
   Token(form='『', tag='SSO', start=36, len=1),
   Token(form='작별', tag='NNG', start=37, len=2),
   Token(form='인사', tag='NNG', start=39, len=2),
   Token(form='』', tag='SSC', start

In [42]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [43]:
nouns = noun_extractor(text)
nouns

['김영하',
 '살인자',
 '기억',
 '이후',
 '장편',
 '소설',
 '작별',
 '인사',
 '미래',
 '배경',
 '소년',
 '여정',
 '유명',
 'IT',
 '기업',
 '연구원',
 '아버지',
 '평화',
 '철이',
 '수용소',
 '날것',
 '감정',
 '혼돈',
 '세계',
 '정신',
 '신체',
 '위기',
 '직면',
 '동시',
 '자신',
 '사회',
 '배제',
 '처음',
 '소속감',
 '우정',
 '철이',
 '수용소',
 '탈출',
 '여정',
 '질문',
 '세계보건기구',
 'WHO',
 '팬데믹',
 '선언',
 '작가',
 '작별',
 '인사',
 '개작',
 '분량',
 '원고',
 '주제',
 '인간',
 '인간',
 '무엇',
 '인간',
 '인간',
 '존재',
 '경계',
 '어디',
 '소설',
 '계속',
 '가치',
 '세상',
 '만연',
 '고통',
 '질문',
 '이야기',
 '팬데믹',
 '개작',
 '영향',
 '원래',
 '작별',
 '인사',
 '구상',
 '맹아',
 '개작',
 '발아',
 '그것',
 '작가',
 '제목',
 '마력',
 '자기',
 '이야기',
 '느낌',
 '탈고',
 '얼마',
 '원고',
 '애초',
 '생각',
 '작가',
 '전면',
 '수정',
 '작별',
 '인사',
 '작별',
 '인사',
 '시놉시스',
 '초고',
 '정도',
 '김영하',
 '이전',
 '문학',
 '세계',
 '연결']

In [44]:
text = ' '.join(nouns)
text


'김영하 살인자 기억 이후 장편 소설 작별 인사 미래 배경 소년 여정 유명 IT 기업 연구원 아버지 평화 철이 수용소 날것 감정 혼돈 세계 정신 신체 위기 직면 동시 자신 사회 배제 처음 소속감 우정 철이 수용소 탈출 여정 질문 세계보건기구 WHO 팬데믹 선언 작가 작별 인사 개작 분량 원고 주제 인간 인간 무엇 인간 인간 존재 경계 어디 소설 계속 가치 세상 만연 고통 질문 이야기 팬데믹 개작 영향 원래 작별 인사 구상 맹아 개작 발아 그것 작가 제목 마력 자기 이야기 느낌 탈고 얼마 원고 애초 생각 작가 전면 수정 작별 인사 작별 인사 시놉시스 초고 정도 김영하 이전 문학 세계 연결'

In [45]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('원래', 0.6794),
 ('세상', 0.6729),
 ('이전', 0.6568),
 ('어디', 0.6205),
 ('원고', 0.6179),
 ('애초', 0.6105),
 ('김영하', 0.6049),
 ('팬데믹', 0.6007),
 ('전면', 0.6001),
 ('존재', 0.5995),
 ('인간', 0.5976),
 ('자기', 0.5787),
 ('초고', 0.5772),
 ('탈고', 0.5747),
 ('이야기', 0.5726),
 ('정도', 0.5673),
 ('정신', 0.5624),
 ('처음', 0.5576),
 ('세계', 0.5512),
 ('시놉시스', 0.5475)]

In [54]:
print(df_unique['translate'][6])
nouns = noun_extractor(df_unique['translate'][6])
nouns

동전 사물함에 유아의 부란 시체.


['동전', '사물함', '유아', '시체']

In [53]:
text = ' '.join(nouns)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('유아', 0.881), ('시체', 0.5597), ('사물함', 0.4752), ('동전', 0.2498)]

In [57]:
df_unique['keyword'] = df_unique['translate'].map(noun_extractor)
df_unique

<ipython-input-57-8f68586a727d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['keyword'] = df_unique['translate'].map(noun_extractor)


,district,address,info,date,full name,latitude,longitude,translate,keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살]
...,...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음,"[심리, 하자]"
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시...","[심리, 하자, 설명, LIFULL, HOME, S, 에이블, 카미이시, 카미이점]"
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견,"[자살, 시체, 발견]"
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택,"[타마, 뉴타운, 영산, 단지, 호실, 특별, 모집, 주택]"


In [58]:
def get_topN_keyword(keyword):
  text = ' '.join(keyword)
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
  return keywords



In [60]:
df_unique['Top 5 keyword'] = df_unique['keyword'].map(get_topN_keyword)
df_unique.head(10)

<ipython-input-60-9c85fb9f6d77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['Top 5 keyword'] = df_unique['keyword'].map(get_topN_keyword)


,district,address,info,date,full name,latitude,longitude,translate,keyword,Top 5 keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]","[(발견, 0.8794), (시체, 0.8008)]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]","[(사우나, 0.9334), (로얄, 0.7064), (발견, 0.6721), (시..."
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살],"[(자살, 1.0)]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,20층\n심리적 하자 있음,"[심리, 하자]","[(하자, 0.8549), (심리, 0.8209)]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,동전 사물함에 유아의 부란 시체.,"[동전, 사물함, 유아, 시체]","[(유아, 0.881), (시체, 0.5597), (사물함, 0.4752), (동전..."
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,902호실에서 자살 있음.,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,이세탄 테넌트 관계자의 목 매달아 자살,"[이세탄, 테넌트, 관계자, 자살]","[(자살, 0.7929), (이세탄, 0.6237), (테넌트, 0.5543), (..."
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,309호실 목 매달아 자살,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"


In [ ]:
df_unique